# Text Generation with RNNs
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F




### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
'''TODO: set the path of the file'''
path_to_file = 'Shakespeare_karpathy.txt'
text = open(path_to_file, encoding='utf-8').read()


#### Inspect the dataset
Take a look at the first 250 characters in text

In [3]:
print(text[:250])

That, poor contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, kno


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


62 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [7]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'That, poor co' ---- characters mapped to int ---- > [29 43 36 55  4  1 51 50 50 53  1 38 50]


#### Defining a method to encode one hot labels

In [8]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot


#### Defining a method to make mini-batches for training

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [10]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [11]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        self.rnn = nn.RNN(len(self.vocab), self.n_hidden, self.n_layers, batch_first=True, dropout=self.drop_prob)   #batch_first=True -> input and output tensors are provided as (batch, seq, feature)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))

    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''

        # Passing in the input and hidden state into the model and obtaining outputs
        # x of shape (seq_len, batch, input_size)
        # hidden of shape (num_layers * num_directions, batch, hidden_size)
        out, hidden_t = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)

        
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden = torch.zeros(self.n_layers, batch_size, self.n_hidden)

        return hidden

In [12]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        self.lstm = nn.LSTM(len(self.vocab), self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)
        # add Dropout layer?
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))
        # add sigmoid layer?


    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        #print('lstm in', x.shape, hidden.shape)
        #cell = self.init_hidden(hidden.shape[1])
        out, hidden = self.lstm(x, hidden)
        #print('lstm out', out.shape, hidden_t.shape)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        #print('linear out', out.shape)
       
        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        # cell state should be included as for now the model only backpropagates the loss over the hidden states and not the cell states
        return (hidden_state, cell_state)



#### Declaring the train method


train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [24]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            model.zero_grad()
            #print('inputs', inputs.shape, torch.cat(h).reshape([model.n_layers,batch_size,-1]).cuda().shape)
            '''TODO: feed the current input into the model and generate output'''
            print(torch.cat(h).shape)
            print(torch.cat(h).reshape([model.n_layers,batch_size,-1]).shape)
            output, h = model(inputs, torch.cat(h).reshape([model.n_layers,batch_size,-1]).cuda()) 
            '''TODO: compute the loss!'''
            loss = criterion(output, targets.view(-1).long())
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    #print('Feed2')
                    '''TODO: feed the current input into the model and generate output'''
                    output, val_h = model(inputs, torch.cat(val_h).reshape([model.n_layers,batch_size,-1]).cuda())

                    '''TODO: compute the validation loss!'''
                    val_loss = criterion(output, targets.view(-1).long())

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                '''TODO: sample from the model to generate texts'''
                input_eval = 'Dear'
                print(sample(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data

In [32]:
def trainLSTM(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            model.zero_grad()
            '''TODO: feed the current input into the model and generate output'''
            output, h = model(inputs, h) 
            '''TODO: compute the loss!'''
            loss = criterion(output, targets.view(-1).long())
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    #print('Feed2')
                    '''TODO: feed the current input into the model and generate output'''
                    output, val_h = model(inputs, val_h)

                    '''TODO: compute the validation loss!'''
                    val_loss = criterion(output, targets.view(-1).long())

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                '''TODO: sample from the model to generate texts'''
                input_eval = 'Dear'
                print(sampleLSTM(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [14]:
def predict(model, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    #print('predict')
    #print('h',len(h))
    #print('h[0]', h[0].shape)
    #print('Feed3')
    '''TODO: feed the current input into the model and generate output'''
    #print(torch.cat(h).reshape([model.n_layers,1,-1]).cuda().shape)
    output, h = model(inputs, torch.cat(h).reshape([model.n_layers,1,-1]).cuda())

    # get the character probabilities
    p = F.softmax(output, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h


In [33]:
def predictLSTM(model, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    '''TODO: feed the current input into the model and generate output'''
    output, h = model(inputs, h)

    # get the character probabilities
    p = F.softmax(output, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h


#### Declaring a method to generate new text

In [30]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    #print('sample')
    #print('h',len(h))
    #print('h[0]', h[0].shape)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
      '''TODO: pass in the previous character and get a new one'''
      char, h = predict(model, char, h, top_k=top_k)
      chars.append(char)

    model.train()
    return ''.join(chars)

In [36]:
def sampleLSTM(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    #print('sample')
    #print('h',len(h))
    #print('h[0]', h[0].shape)
    for ch in prime:
        char, h = predictLSTM(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
      '''TODO: pass in the previous character and get a new one'''
      char, h = predictLSTM(model, char, h, top_k=top_k)
      chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [16]:
''''TODO: Try changing the number of units in the network to see how it affects performance'''
n_hidden = 256
n_layers = 2

vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)
lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

VanillaCharRNN(
  (rnn): RNN(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)
LSTMCharRNN(
  (lstm): LSTM(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)



###### Declaring the hyperparameters

In [17]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 10
seq_length = 50
n_epochs = 20  # start smaller if you are just testing initial behavior


##### Train the model and have fun with the generated texts

In [217]:

''''TODO: compare the results from the vanilla and LSTM model'''


"'TODO: compare the results from the vanilla and LSTM model"

In [18]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/20... Step: 50... Loss: 3.3610... Val Loss: 3.2876
Dearet l t e er  so o estenooies atslos   r   ot ese l rooeren r  s niootosle o a l   e  rniear lil rrr elnisea soor olioienet n  ersnrtsseoeetra eilet oesir  s tnar ense   leetiieto aotieeee   ere ei se rrr eorloie   ni  ro ol lnee etor ora aaeer en arretntrn naolaseooeeaenss no srseoo ornao etst a otrenla lon i ese ooaio  alr iitsrssel    aoiotent letlnsaoneae eetrnr tteaoet  ste oootaen rie seelnetao lnl sto n otrseer  lo saniinroo tna  eaaoirsaesetoa neiaioeneiananss  ii ee eoa e troie  nins re r s oes  i rtee sleaerrster s elaanleeoleet rii sie  ate eior tl iee  eai ire roee lnaoroi  ls lerren   leiaa ote  rtteln  i eootn l es  iesnetai aisteo eaiettelaottlinse eoeoisallai   aiaeot tleio ei oenooorntestra as rttottelnae oii to eolslnr rttatos  aeir r   n si  ano nne ensr rs oar astoisao   eellloinn ie  ii erese lt rraaslaa     len srtssaoto asto ar loe as a taae lea oanr llosso  srlta i olett ete na eaon to rote e  eo rne

Epoch: 3/20... Step: 450... Loss: 2.1599... Val Loss: 2.2080
Deary here wrond amy lost on thy ut sash wery wrerther bors; I whur ander, I what if thill,
Bfteachad, and heat, hilidt ow shererer this, ther thar mine are hon to apind;
And you hy by int he mane, an spome you, wy and, agd thacs, the hene tidirnd ond ther miss my wathorers bon to shonds.

CASERON:
I masheth whang, morent share:
And ond will mids ast hit, a bistre werent,
The state, to beace stailt ssot tores batser thay myay woll, sit tile; the this ther mace ousees.

TINI:
As thet ar be seritsiee thir the trithee tolt as his to tome, shat hin theses indsenst in the hon to ther his, as cowald you core ther ther hanes, ther, wather an manse,, sher of lott tithar' at heme at on, hinstreet tract your sout most is as satssens of sand,.

TITUAR:
I sist aglthint blone ant you thes,
Af his wasteren't ou sirstort, as merest il thendis it here,:
There, thane he hen, of the sous, sor mind be wreir, oot hered sat is trin,'s shent is it

Epoch: 5/20... Step: 850... Loss: 1.9334... Val Loss: 2.0389
Dear
Wour: a camy, a wires,
Ther iselishe
Othele thatl thy elires:
Wardont wirhs ar why sweng with twe krack to dat tree is mantus heme ot thamis, tor wave.

SILER:
He fasseld o tolensed cowengrees; of the wanced oust this bet tore, that dadiont it thace,
Tule douth me be the hive
Wronthor a hese,
If swoul as not ther antich to sagrimen to bistrem.

CILANLESSA:
Mare,
Ald thas the putich day, my beart, a bave inounted a futrent douthy touth, ingant. Sillan:
Andt arate, and bo daspacaing ore one mis,
I thers thind tor gor hisgead,
Toumestlonce say?-
OMTAMENS:
Thatl not hastiaking tome
The secesy is the wired
Anct dote.

SOGNONO:
Wide hame to hastillion,
Bo got
And stelucy tham, and, my torsenos thim thucangest and,
I' mut carsaes, and are a beny, and dooshed the poridirs, and bour and thate be oureliness dean ther,
Weniee,
But to be my aling thes cruinghiligher mintt, and michid, be beeted ther, and asit, there ant, thiness, by

Epoch: 7/20... Step: 1250... Loss: 1.8471... Val Loss: 1.9210
Dear:
By make are in seminct seer, bonk.

OARA:
And mord, in the gide wir to meny all sens your andither, and the gentie custom comfsly mardion
Will but have she lord.

ORASSIO:
Herr love wo to the with me sthen bl not he fornoss and will such fen thou that that bale
Thees sins me the selines.

AAGE:
Ho kell werl combowe sen we houd he tone as, feine, am latine.

OLONAE:
And make it with a weivion
So all wo ferels in the gath will me hoth seem, theat say a mungend my make
The full thing be their hould and we cring so mightarss, whis floct;
In mony hough to see iligy wo co fitch of tum of lath to fare betine, the was it on she ford fryiod her be to streak's shall shall say, with a sale inss fring forder,
To ther's centum consen brang thus heat hell.

ANGACO:
Ay thos gade; the camy: that thy has.

SANSARIO:
Allow him should ame lodies and tunt, born to hant, this fean thus baselled of serpion
That heve in my lond;
She ment the

Epoch: 10/20... Step: 1650... Loss: 1.7437... Val Loss: 1.8597
Dear I wast of tle bears so would so the showll shild the hond, she cut my'd bechy at are and gries wister ands so his his hich mass;
Trok not won a minon with a prrsest buse.

BATILESS:
Wo mur tho ashers in they strees to meg as me, but hhis feed
Hurbs bound my such
That they worrely were their good then his hand have me as I pidst thoughtly and the say a sweets of year
Things all thee.

PEOTAS:
A choust me bronker, I would taly not hem, sir, she mutenald; if lokes lise of you, hroware my deed but shove
Thum spirper'cadest within speet brinch with swreegh with the sard that senss, that wan the sue, I keep to siends,
Meaver herd be dother that she porsean apan stert, his canded,
Whtare on sooked him should tell byor, that sweep,
Sne seeplet. Tountaliand brontred sit offers a great as wiled he, hur.

SIL HEARNA:
What his wands me hy lord,
She hever bo so most here;
And ho trest my lang'gain so bless.

TRANTOR:
Thy stome.

BE

Epoch: 12/20... Step: 2050... Loss: 1.7896... Val Loss: 1.8150
Dear thrive so me their swartest of hanown arain,
When thee: ape feardy
And be world thou shall nay in my my sie.

ThINT:
I his fold not dessles, that thee nother alfors it never:
But his flese,
As my, compose. Bound and that your freture to the lade.

Frostres:
Whoth as one hapt tuthers.
Whathing her
For me thene as the conowe youn thaigion of mone.

TIMER:
Therefore do sire's wishos could as me betrution;
Or my proud
The firmon and till as see think mineter, shilist straight
Therefore in your follow, ou look, what can do to your hant have.

KING LEAR:
To cersentide thou art tame dond, seeving sacr.

KENG LIURENO:
I'll bear with offer,
Firloon world his father'd,
Whemer herpears,
Thee a mis lord,
Tith with mish me precentreans their a many they, I twos from her dord to by since, wn manes, and proves, and a signd.

DUKE VINN LUSEANA:
Be fouse till well he way therefore. They live as she litter?

LUCEUNEA:
Ho love mine will.

Epoch: 14/20... Step: 2450... Loss: 1.6818... Val Loss: 1.7848
Dear, a spacion; their here,
Wo dide it some her do have in heard to comet of hertly, my moody be such and for twee, are of lotise this helate
Thear werter what we was ret her heaver
And heath, by tid the megare
to my before the stains it in the deasted, it a grain and ther, that soch able, their deadles
Wyeir themed'd have and fall neard of them metiee, if my teeling the fortion, iun's deeds and badis well not, I would no stayt mear till plise
To foild thee a brow thee speaked alonaly hadreate:
How all hers a sace
Than thee: for caum'd steer
Ar dlues:
As had shee in thee,
What
stere,
And with stecce
Of love,
Who make and grace as you ars is to mone he say:
The treet to shall ht for the asiass and mains as they me mest thou tair hopf it the grove at you as the weeteed for my tray, by twinks,
A treet souly a krees whit ope that your granct them too,
The futes wo bees
To the pease of bress.

Second Servant: thy, wheches, and 

Epoch: 16/20... Step: 2850... Loss: 1.7520... Val Loss: 1.7712
Dears we chather, and, shriff and to hold the lood,
Tell my lurd,
And thressing;
Mest be maken on my lerd.

DAVIO OF BORK:
He hose mather tin thou art fortume
there sawith, a forlt thee.

KONG LEAR:
O makes
And well, I will grace:
Sire think when's like as you wile shall me with the creace
My lire; and ghe levand
The will,--

TIVELIUE:
Then'es with the doble my mune.

Second I whell his been'd my hays tunthe thou shame is he say them deart,
Which we canlant, what:
Thou arr.

KICGIDES:
Tell sisheart him betore the say, I meriness and wirr tedy too trungen, and streace we love are deach
And stousice of a gremant our death at when, I have allame to me so tell you and with a truend: my lord!

CESSAR:
That grace;
They with me to misted he well precoor' thus a may you have suchise
That thou should like and fiest;
Benger his,
Then by ags thes is the dcit thee namr,
Hindous giteed
With my hight as a woml, and flom who camoneroce on

Epoch: 19/20... Step: 3250... Loss: 1.6316... Val Loss: 1.7523
Dear, if at yeme a maie, hand to be with most make it for fair my think from ut. But with the pat and a wilk ne make their dobe that tume him we shall a come, should I, fay foldowerly on the warth.

PLODY
ANAES:
Hith will mestar''d abt our discerow may would hame in dise as this good me the piemed, and a glieve my prome.

DORETIA:

TORTIA:
If, tho kevery. How marry, have that, I am we hath stope
To dignt hispone your and stomm as and such offectias to my lord, I do wet
In that'd fall thou no herp hope of heart. Beisting of my man was amblion,
Whose satious agnoud of the purson'd say! That a feart
To begower, she sheet
As therewore the pition, I am causif in the brom at a wurester we crieds
He hearme the heard,
That as trom's a gone.

CLIUCIO:
I am a flumber me make my deed.

BARDUCONA:
Orily blow of thealt
Be whim him at it as op miscuce
Anon the maiten and nobles, I pead,
On thy would now we mine is that as twell straight 

In [37]:
trainLSTM(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/20... Step: 50... Loss: 2.8601... Val Loss: 2.8661
Dearst oen ehos s te aa soret
aeoe  asen e osolad 
iitd ninrnea
lotle totos oie hote isnsesnls
hhtr tit ersst hn e rae ro erasaar 
ontt soe meeth tinilns
 oaere hta sod  oe aa s ma o re setht rrllt hol anr ots mttiee 
rtoe tne aarinet itomed aaosr,the taoit tie rhilrl n e se tre ee ns te ttoo h oae 
oe lrer n roia tollra

ohi nsssnet rit iis te raaa eienn tirrdr
oe ese aenlldr oo sesd nil aele na neis aae tnle tthel ie sai shord thtrnns ialrn rr ro teni hare aeer eel nhe ee no ies to i aaon satl aatd 
ttae  sae aaol rtrasnt se sede  oonlt neerd se an eh noeee s toia hsrld r hhe eoe heeer to r tie mo se oe to e at iiltrl ate tertoae to ms aaoered nond nstr aheas sientrsrs mos etnsthti nene nir hi rea
at tateert ea iet hon aoite 
oree


OASSAS

U
EI
IO:
I oe rras e toar niasd mn nonarsnl ri sanrs seatlas tor reot oe rai sees hi toorte ee itr aoane nr hent aor eors aaote 
atter oheed ooen oe toredt
ee ta rt e soie mto ea aritrdr 

Epoch: 3/20... Step: 450... Loss: 2.1411... Val Loss: 2.2205
Dear, I to thes, as aud an som he hishs torteete mentirer ind you cavyint wo comare witt thrun he wes this hant is barld af met os the wice
Way watore, an andenssid'ns than he theserend thy wamd by his aride, an bindeand of, to wive
Sou brenot,
Sut lut ye wald;
Afd oud I her sus werle.

LASTONUS:
That mislames tho thavis sare so sangs,
Mis hilgis hensen mondicin hamd hars, then, so tise houd yin, seracith ilg stouce by bant.
Thit mo thir sivt ceametsith,
Widtir of hid of the beinis out my his syome chome wo che fares'st wis theast;
So thas to gonterte, I my weren, of af it the heer, I thee by thill, of me bace tattirn thalish is thee, at his ald tous yit theus athens ot th hers, and busderith, wo momt at.

CKLANEI:
O thourd thes yours ar andten, trave hondeteser thens ond sas thithat;
Hhating of sante,
To thee ang sorentent in on my thivid be as thisg berer thang by in

furtortoud, anos on therend we cunntot,
An in ar sord, t

Epoch: 5/20... Step: 850... Loss: 2.0020... Val Loss: 2.0630
Dear hpors, they apeire
Asters werd, bit sheseres are
Warceriin harsuls, and mlcecand sare.

TLEUSHON:
Oy bucand a wror to sheaves agenalid
Theresall apeath menith mestos hin
Anderingiss,
To sonded and
Were ou truid to dacess owery.

COKUSONIA:
I haldolou theace toons athoused
Afalled thyar har, as the wisgor the galcinge,
Be dimy'r oftues meed to wher andy;
Thanghing thousd blattan, andire.

KOR PEENUNRELI:
Tiy, afer nom shone buvere beticith in of nele;
And dond, thou beas to day, tore, farer, whe wall, as stolant
Outhow shan at is thas whing sall oul my wisgead bivrich thered
Settlesher, all ow bese wouthinger asend,
Ir wo bas of theas hame.

TARK:
Ard, I masteres shiths with shamid sarpentele;
And were wemte opens beat
On mant in to crutoulsed and sheitht of assing sould my alroo
Oretince thy callest
Worders meanit he mare as then, I wisht.

SIANGIOD INY::
Sereed sie, it hame oursterites.

SIRTINTUL:
A,
Howed srow, and it

Epoch: 7/20... Step: 1250... Loss: 1.7985... Val Loss: 1.9369
Dears wishe ment word, as sees to pane when most for borne;
We, alimn in my mant, and mider beden me
Ald not mathous fith a beting, yit leve and hore.

BICONIA:
I, wistrend be shald thy lide thy hears brouglt it you srore?

MORTACINEND:
I that dowiroun tall de the dath offand so strents, with
Of hive there and my lost, and and hist bet and angy, I wowld wo couth and brain whe pays this shour migr thy lither srowe,
To pue and aml and, I cameders.

ONIONA:
Sow highos sor, that with wist still wear hive and to hale
Tood halime and sweme bund to beand, we with is which amy made ame
And heart my wish, me losk when theme, till bet thee
The duth staight sill be mider ow while bughand bind, we murier.

ANTATANS:
The mad as eat, and his sreces, fith whough bees shell.

KIGG CHIONRUO:
She dom stertiladint'd sonow
As it we sharg a ame asted thee, we laveres bowe shall for a marlt toore.

SISONLO:
Sas all teme agpotince sach to shive me

Epoch: 10/20... Step: 1650... Loss: 1.7493... Val Loss: 1.8674
Deard,
And me mull oner a belore your theep'r,
What son of mont the love.

DUKE AF EFALY:
That, I am a way your all west a sponour stanses.

SLAST:
These loke, and my dake that his a mpillate, sheme as geliin
Shougrss must these saksionst angood his. Some they thneem ald
Terping home; I am dose be and my
Angore we thy to the sak his tige that stoll of for they but tto him there a penty of as meer, and bughers.

ANGELE:
As she lover him not fremore, headth wist.

MIRGARDIO:
Os you whand nos, which a sworke with seer broik so from of a but spenthed songer,
Wher those prothincs,
Hake mung so be and hid say, as the lorgs or gonder's.

CLESD HARBATIO:
The lest this curd so to thise my sonth myselp, and to though frese,
Mngos wist thut a deed a whas's sommure to-chimg the gruin a folr.

ANTOMALLO:
Serfuld tricchas my flind his son wall sentham my tates, and my love
Frath she hose hopderst of me but with will not comfert.

MARLIAN

Epoch: 12/20... Step: 2050... Loss: 1.8492... Val Loss: 1.8077
Dear to ance of in her
stace you hand. I thou toot amath, and me sure,
And leave whor haspy hers, it thy flond.

BRETOUS:
Thou thoull my copt,
Ay some; therelope there aboing, he stakent of though of me.

KING LEAN:
What, this to youllly sowerour, in thou dos in the llomy;
That here here, and thought. He this but of man he chall th in my live it for to me with
The lorg be so madiers to shall be be hind
As a shes, he be leod of the cereats.

DOCTLAPIT:
I will make your bleader it: apter to thee:
And here to my sone to this and and the poor consence
Trire tinemind have be sword, and waret hear maress with the
This hid bedent on theer siemas, whith withing,
For was with home, and whith beart hade here shere wet me themers; arland and tome ane sone all meer, my hould and mo.

CELEPTIA:
I what it mather her sich thouk that to befloves,
Hath not thee wath be arou sow me
Fas, it il to consers;
We lord, mun with the beaich must thy

Epoch: 14/20... Step: 2450... Loss: 1.6391... Val Loss: 1.7699
Dear falr of freised in ever
To bear my dinch hermasent see, if thene lus begusent thou sumstence
To faie not bungan theil ant the patthers
Wring wourt thou graited to plusty that her them.

ANNE LUGR:
Nor: and would be sage by ours, a dot stee, and be penfeats,
Which be thee hear, and these thy heabts of break now no gold
That man my lord from the tent and good and as hes laders,
And mer the censters of a madian
That signint bry her are beer, and, who withly
whist see them were thee master sight
As so my bear to you a weas,
And fill eptest we'll trum in my masts.

KING RAIDINE:
Tarr's looked, for fermen meselcence ane my monser with her beace.

SRISSIUS:
He most, for you are not fall no gid these parith.

DURI BOLINT:
What should, who have her homess to came tige to hear some betiniss;
And badd my mides in out is futher and suble
So frott a feer of my meaper hamself,
Ofe a froble not to the saget while well to make hak me.

Epoch: 16/20... Step: 2850... Loss: 1.6567... Val Loss: 1.7377
Dear ard by liteld strack though thee.

DESESTIO:
I seis but and theight so for a strime:
Where perfor him as a grace that I miret,
For, ther,
So stand he would not the gass to him, thou thum tree:
If you be so the purction; which men liffor
Without aspeit my from the tongun'd for ouls should:
And icencious mate of men bunder and here ofe
A would pespont'd of than would prove a true a host his sower
The heart a sigh for, the thouse
Bu kues of a mece on the woncus make me hamst,
To for as these prame and good was hath palice; if we broughts:
The suckt the same you a chirde to the fith them dight and, I have
the world of shall sweed it?

CAESRINE:
A, I womld not place thou hope me that sween.

PIORLEL:
Os he doth the place with whing for me.

Thind Cintllmen:
Is we all'd yut ane but and
That with having well before when: my distreichtule and the gied out be thy steat.

KING JORN:
Sheme then before in the say?

KING JOHN:
I, l

Epoch: 19/20... Step: 3250... Loss: 1.6063... Val Loss: 1.7161
Dear you mare
Men blook this any to sem thear her such at fings.

PALANIO:
She'll not her befuching me house better,
Or I would not did to content and mares,
Too to ground hatt diused bosne my: by my fance,
Her lodging to a spife.

DIMADDO:
He have meer fhat o't man to me, and be she stornions,
With nwrest it with thy tumer withit thy wear in a geent
But thou hart for their same
To stain,
To good dies with will not mane all a mrouth,
Whose sames me; are with the loddisise,
If thou must thou to see, a bring of his his such as the pitiles
To make himse in the brother as her ase pear thou art,
And thou come the master must from your loves sich
My hersen a gurture.

BOATAN:
Shat I am the parturubors, a fair aram, her any have it sheard in my tould
With her string, well this all in the have
That to make a true, if thour were a poin'ss baul
Hen should to this will and any
these ally master, and my lord
And ther in me we fairs: bu

##### Generate text

In [38]:
print(sample(vanilla_model, 1000, prime='The', top_k=10))

The flame.

OLANTIOL:
O would be we wall
I see hers sue, when seent it of dafe,
And the from my sword in which sibrer way word a from but fors thus shall bear offect be your stren's me the hath oped, I was gone that to the woined me what I spoul,
An I suw the sevan of a fuch and sat of me would sweat the foo callow whie sthen
I stock?

MARCESTER:
Hing an excotienders that I would not lease
We wouth well sin witles shall be,
Are imelly take the frame me, I'll precorraw'ds all any the well; I, swort
Ho lost. Here not but the call you would hell me thlich the guold
So was,
And marrow, swerilour;
Whone dose a musbeet callant.
An losh
Of tho were that see? serving
Well; beacly a for of men to be go thy thus she wasce the straibled in well.

CRESTIO:
A,
Tell show in: see should born, what it wercusom be to whing lalked.

MACKINO:
All I do not am while shall not when I love, I would see, as see a good confree a gites you well;
And I was ad I must they in thy daffall all thine the gright of su

In [40]:
print(sampleLSTM(lstm_model, 1000, prime='The', top_k=10))

Then thear sigh of thee:
And sword you would to the himper some ang it?

BRUTUSUS:
I whis thou host of all take a dally
From thou eptaces of do stare, and tell wo have stict.

OIMOL:
Hearm,
Ill fear that would bet yet; that I censteriin fault with their hangs,
Hew me hath not deas intine the gore.

PRONCESGUS:
Be to-not with him but, what is, my prace and see what
shall my seeving me,
Look what grough buccomes of have as give a moy a semest would
Tell the liged with thein shall stand his true.

MARK ANTIAGO:
Stirl of her for the with sich that thyse sagries and base the monsuan
As greace them for whither
To confearos, the will for a subeliins have in
my fortused a fham with. Py our shall
And he do not gongur, with him, I am my stay;
But is the fire of twenk, sir,
I wide break your crarmest and livery sone
To the chirgiaged are, that he pear the wise sony
I shall, and I would no love of her wind their give
But hath grive my foor and all their fires,
Whil should now, who me as stalision 